# Предобученные модели

На Huggingface доступно множество предобученных языковых моделей. 

https://huggingface.co/models

Следующие модели содержат русский язык.

## Сбербанк ruGPT

ai-forever/rugpt3small_based_on_gpt2 __125m__ \
ai-forever/rugpt3medium_based_on_gpt2 __356m__ \
ai-forever/rugpt3large_based_on_gpt2 __760m__ \
ai-forever/ruGPT-3.5-13B

## Сбербанк mGPT

ai-forever/mGPT __1.4b__

## Huggingface BLOOM
bigscience/bloom-560m\
bigscience/bloom-1b1\
bigscience/bloom-1b7\
bigscience/bloom-3b\
bigscience/bloom __366B__

## Facebook OPT
facebook/opt-125m\
facebook/opt-350m\
facebook/opt-1.3b\
facebook/opt-2.7b\
facebook/opt-6.7b\
facebook/opt-13b\
facebook/opt-30b\
facebook/opt-66b

## Meta LLaMA
meta-llama/Llama-2-7b-hf\
meta-llama/Llama-2-13b-hf\
meta-llama/Llama-2-70b-hf

## ... и другие


# Загрузка предобученной модели с HuggingFace

Выбор размера модели:

Допустим модель имеет 1b (миллиард) переменных

На диске она будет занимать от 2 до 4 Гб

В памяти GPU она будет занимать:

* 4Гб с опцией **torch_dtype=torch.float32**
* 2Гб с опцией **torch_dtype=torch.float16**
* 1Гб с опцией **load_in_8bit=True**
* 0.5Гб с опцией **load_in_4bit=True**

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# model_name = 'ai-forever/rugpt3small_based_on_gpt2'
# model_name = 'ai-forever/rugpt3large_based_on_gpt2'
# model_name = 'ai-forever/ruGPT-3.5-13B'
model_name = 'ai-forever/mGPT'
# model_name = 'facebook/opt-30b'
# model_name = 'bigscience/bloom-3b'
# model_name = 'meta-llama/Llama-2-7b-hf'

model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model_size = sum(t.numel() for t in model.parameters())
print(f'model_size {model_size/1000**2:.1f}M')

model_size 813.6M


# Генерация с префиксом

In [11]:
prefix = 'О сколько нам открытий чудных'

inputs = tokenizer(prefix, return_tensors='pt').to('cuda')

outputs = model.generate(
    **inputs, 
    do_sample=True,
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    )

print(tokenizer.decode(outputs[0]))

О сколько нам открытий чудных. Пора от этих открытий вам открывать новые возможности. А вам счастья.
Добавлено: Чет 30 Августа 2006 16:40
Даша, вот ещё из детства, такую подругу... с нами не считалась, а у меня одна была - и вот такая! "Счастливый ребёнок"
Добавлено: Чет 30 Августа 2006 17:16
ДД! Давайте об


# Параметры модели

In [12]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(100000, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear4bit(in_features=2048, out_features=6144, bias=True)
          (c_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear4bit(in_features=2048, out_features=8192, bias=True)
          (c_proj): Linear4bit(in_features=8192, out_features=2048, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elemen

In [13]:
model.config

GPT2Config {
  "_name_or_path": "ai-forever/mGPT",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 2048,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 2048,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": 